In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path = '/kaggle/input/ubiquant-market-prediction/train.csv'
parquet_file = 'train_red_mem.parquet'
train = pd.read_csv(train_path, nrows=1)
features = train.columns.tolist()

In [ ]:
features = [f'f_{i}' for i in range(300)]
dtypes = {'row_id': 'str', 'time_id': 'uint16', 'investment_id': 'uint16', 'target': 'float32'}
for col in features:
    dtypes[col] = 'float32'

In [ ]:
def write_parquet(path, feats, filename, chunk_size=10000):
    schema = ''
    writer = ''
    if len(feats) > 0:
        for i,chunk in enumerate(pd.read_csv(path, dtype=dtypes, iterator=True, chunksize=10000)):
            if i == 0:
                schema = pa.Table.from_pandas(df=chunk).schema
                writer = pq.ParquetWriter(filename, schema, compression='snappy')
            table = pa.Table.from_pandas(chunk, schema=schema)
            writer.write_table(table)
    return None

In [ ]:
start = time.time()
write_parquet(train_path, features, parquet_file)
end = time.time()
minutes, seconds = divmod(end-start, 60)
print(f"Time taken for converting csv to parquet: {minutes} minutes and {round(seconds,2)} seconds....")

In [ ]:
train = pd.read_parquet(parquet_file)
print(f"Shape of train data: {train.shape}")

In [ ]:
train.head(10)